In [ ]:
import os
os.environ["DISABLE_MPI"]="true"
from fgbias import setup_AAAA_recon, get_bias_terms, get_cl_smooth
from falafel import utils as futils, qe
import numpy as np
from orphics import maps
import healpy as hp
from pixell import curvedsky

In [ ]:
# Set some options for the lensing reconstruction

In [ ]:
lmin = 100
lmax = 1000
mlmax = 1000

In [ ]:
# Set noise level etc. to get appropriate filter
noise_sigma = 5.

beam_fwhm=1.5
ells = np.arange(mlmax+1)
beam = maps.gauss_beam(ells, beam_fwhm)
Nl_tt = (noise_sigma*np.pi/180./60.)**2./beam**2
nells = {"TT":Nl_tt, "EE":2*Nl_tt, "BB":2*Nl_tt}

#total noise 
_,tcls = futils.get_theory_dicts(grad=True, nells=nells, lmax=mlmax)

Read in foreground-only alm and get power spectrum (we'll need it later)

In [ ]:
fg_alm = hp.read_alm("/pscratch/sd/m/maccrann/cmb/fg_outputs/cib_fcut7/prep_map/cmb_orig/fg_nonoise_alms_0093.fits")
cl_fg = get_cl_smooth(fg_alm)

Setup lensing reconstruction stuff

In [ ]:
px = qe.pixelization(nside=4096)
setup = setup_AAAA_recon(px, lmin, lmax, mlmax,
                tcls,
                do_psh=False, do_prh=False, do_psh_prh=False,
                profile=None)

Get foreground bias estimates.
We need a few more things 
- true lensing potential alms (use this to calculate primary bispectrum)
- Two versions of the same unlensed cmb realization, one lensed by the true lensing potential,
one lensed by some different lensing potential.

In [ ]:
kappa_alm = futils.change_alm_lmax(
    hp.read_alm("/global/cfs/projectdirs/act/data/maccrann/websky/kappa_alm_lmax6000.fits"),
    mlmax)
Ls = np.arange(mlmax+1)
phi_alm = curvedsky.almxfl( kappa_alm, 1./(Ls * (Ls+1) / 2) )

#cmb lensed by true kappa
cmb_alm = futils.change_alm_lmax(
    hp.read_alm("/global/cfs/projectdirs/act/data/maccrann/websky/lensed_cmb/lensed_cmb_alm_websky_cmb1999_lmax6000.fits"),
    mlmax)
#same cmb lensed by a different kappa
cmbp_alm = futils.change_alm_lmax(
    futils.get_cmb_alm(1999,0), mlmax)

In [ ]:
bias_stuff = get_bias_terms(fg_alm, setup, 
                   phi_alm, cmb_alm, cmbp_alm,
                   ests=["qe"], comm=None)